In [11]:
import neuroglancer as ng
import matplotlib, os,json
import numpy as np
import pandas as pd
from pylab import *
ng.set_static_content_source(url='https://neuroglancer-braincogs.appspot.com')


In [49]:
# Get segment IDs for the PMA
pma_segments = json.load(open('../data/pma_segment_ids.json'))
cerebellum_segments = json.load(open('../data/pma_cerebellum_region_ids.json'))

In [50]:
# Get dicts mapping brain names to primary injection sites
hsv_disynaptic_injection_site_file = '../data/dataframes/thal_cohort_inj_pcoun_data.csv' # HSV-H129_Disynaptic
df_hsv_disynaptic_injection_site = pd.read_csv(hsv_disynaptic_injection_site_file)
hsv_disynaptic_injection_site_dict = df_hsv_disynaptic_injection_site.set_index('brainname')['primary_inj_site'].to_dict()

hsv_trisynaptic_injection_site_file = '../data/dataframes/nc_cohort_inj_pcount_data.csv' # HSV-H129_Trisynaptic
df_hsv_trisynaptic_injection_site = pd.read_csv(hsv_trisynaptic_injection_site_file)
hsv_trisynaptic_injection_site_dict = df_hsv_trisynaptic_injection_site.set_index('brainnames')['primary_inj_site'].to_dict()

prv_disynaptic_injection_site_file = '../data/dataframes/prv_nc_cohort_inj_pcount_data.csv' # PRV Disynaptic
df_prv_disynaptic_injection_site = pd.read_csv(prv_disynaptic_injection_site_file)
prv_disynaptic_injection_site_dict = df_prv_disynaptic_injection_site.set_index('brainname')['primary_inj_site'].to_dict()


In [81]:
# Single timepoint, single injection site

datasets = ['HSV-H129_Disynaptic','HSV-H129_Trisynaptic','PRV_Disynaptic']
home_dir = '/home/ahoag/progs/pisano_etal_injections'
data_dir = os.path.join(home_dir,'data','pooled_by_injection_site_volumes')

layer_dict = {dataset:['Princeton Mouse Atlas'] for dataset in datasets }

colormap_dict = {
    'HSV-H129_Disynaptic':'autumn',
    'HSV-H129_Trisynaptic':'winter',
    'PRV_Disynaptic':'cool'
}
dataset_shortname_dict = {
    'HSV-H129_Disynaptic':'HD',
    'HSV-H129_Trisynaptic':'HT',
    'PRV_Disynaptic':'PD'
}

dataset_primary_injection_dict = {
    'HSV-H129_Disynaptic':hsv_disynaptic_injection_site_dict,
    'HSV-H129_Trisynaptic':hsv_trisynaptic_injection_site_dict,
    'PRV_Disynaptic':prv_disynaptic_injection_site_dict
}

injection_site_name_remap_dict = {
    'Crus I':'Crus_I',
    'Crus II':'Crus_II',
    'Lob. I-V':'Lob_I-V',
    'Lob. VI, VII':'Lob_VIandVII',
    'Lob. VIII-X':'Lob_VIII-X',
    'PM, CP':'PMandCP',
    'Simplex':'Simplex'
}

injection_site_name_remap_rev_dict = {val:key for key,val in injection_site_name_remap_dict.items()}

active_viewers = []
for dataset in datasets:
    print(dataset)
    viewer = ng.Viewer()
    with viewer.txn() as s:

       # First load in PMA with cerebellum only
        s.layers["Princeton Mouse Atlas (cerebellum only)"] = ng.SegmentationLayer(
            source=f'precomputed://gs://wanglab-pma/princetonmouse_16bit_hierarch_labels')
        pma_layer = s.layers["Princeton Mouse Atlas (cerebellum only)"]
        pma_layer.segments = cerebellum_segments
        pma_layer.object_alpha = 0.08
        
        # Load in PMA with all segments but disable layer
        s.layers["Princeton Mouse Atlas (full)"] = ng.SegmentationLayer(
            source=f'precomputed://gs://wanglab-pma/princetonmouse_16bit_hierarch_labels')
        pma_layer = s.layers["Princeton Mouse Atlas (full)"]
        pma_layer.segments = pma_segments
        pma_layer.object_alpha = 0.08
        pma_layer.visible = False

        primary_inj_dict = dataset_primary_injection_dict[dataset]
        pooled_vol_files = sorted([x for x in os.listdir(data_dir) if dataset in x])
#         for pooled_vol_file in pooled_vol_files[0:1]:
        for pooled_vol_file in ['Simplex']:
            print(pooled_vol_file)
            primary_inj_site = pooled_vol_file.split(dataset+"_")[-1].split('.tif')[0]
            inj_site_origname = injection_site_name_remap_rev_dict[primary_inj_site]
            
            # Figure out which individual layers correspond to this primary injection site
            sample_names_this_site = [key for key in primary_inj_dict.keys() if primary_inj_dict[key] == inj_site_origname]
            print(sample_names_this_site)
            cmap = cm.get_cmap(colormap_dict[dataset], len(sample_names_this_site)+1)
            
            # Load pooled layer
            pooled_layer_name = f'{dataset}_pooled_by_{primary_inj_site}'
            pooled_gs_layer_path = f'wanglab-pma/injection/{dataset}/{dataset}_pooled_by_{primary_inj_site}'
            s.layers[pooled_layer_name] = ng.SegmentationLayer(source=f'precomputed://gs://{pooled_gs_layer_path}')
            rgba = cmap(0)
            color = matplotlib.colors.rgb2hex(rgba)
            # select segments, set color and alpha
            pooled_layer = s.layers[pooled_layer_name]
            pooled_layer.segments = ["1"]
            pooled_layer.segmentColors = {"1":color}
            pooled_layer.selected_alpha = 0.6
            pooled_layer.object_alpha = 0.12
            for sample_name in sample_names_this_site:
                sample_index = sample_names_this_site.index(sample_name)
                rgba = cmap(sample_index+1)
                color = matplotlib.colors.rgb2hex(rgba)
                layer_name = f'{primary_inj_site} {str(sample_index+1).zfill(2)}'
#                 layer_dict[dataset].append(layer_name)
                gs_layer_path = f'wanglab-pma/injection/{dataset}/{sample_name}'
                s.layers[layer_name] = ng.SegmentationLayer(
                    source=f'precomputed://gs://{gs_layer_path}'
                )
                # select segments, set color and alpha
                this_layer = s.layers[layer_name]
                this_layer.segments = ["1"]
                this_layer.segmentColors = {"1":color}
                this_layer.selected_alpha = 0.5
                this_layer.object_alpha = 0.12
                # Disable all samples (merged will be enabled)
                s.layers[layer_name].visible=False

    with viewer.txn() as s:
        pma_layer_cere = s.layers["Princeton Mouse Atlas (cerebellum only)"]
        pma_layer_cere._json_data['setSegmentsGrayScale'] = True
        pma_layer_full = s.layers["Princeton Mouse Atlas (full)"]
        pma_layer_full._json_data['setSegmentsGrayScale'] = True
        s.layout = 'yz-3d'
        s.position = [139.4, 519.5, 271]
        s.show_default_annotations=False
        s.show_slices=True
        s.crossSectionOrientation = [0.7071067690849304, 0.7071067690849304, 0, 0]
        s.projectionOrientation = [0.7109261751174927, 0.7009936571121216, -0.018140649423003197, 0.053505148738622665]
        s.projectionScale = 1024*20e-6

    print(viewer)
    active_viewers.append(viewer)

HSV-H129_Disynaptic
Simplex
['20161205_tp_bl6_sim_750r_03', '20161205_tp_bl6_sim_250r_02']
http://127.0.0.1:34423/v/e8bbb9867e0c8d03c0369d11f6e767280556b68c/
HSV-H129_Trisynaptic
Simplex
['20170130_tp_bl6_sim_1750r_03', '20180417_jg58_bl6_sim_02']
http://127.0.0.1:34423/v/8834884361303044386cafd56198358c9852cb40/
PRV_Disynaptic
Simplex
['20180322_jg_bl6f_prv_27']
http://127.0.0.1:34423/v/1117f114fe4a71691db87a09fbeb9341c6d49c89/


In [82]:
active_viewer = active_viewers[0]
viewer_state = ng.to_url(active_viewer.state).replace('neuroglancer-demo','neuroglancer-braincogs')
print(viewer_state)

https://neuroglancer-braincogs.appspot.com#!%7B%22layers%22:%5B%7B%22type%22:%22segmentation%22,%22source%22:%5B%7B%22url%22:%22precomputed://gs://wanglab-pma/princetonmouse_16bit_hierarch_labels%22%7D%5D,%22segments%22:%5B%221119%22,%22378%22,%22379%22,%22380%22,%22381%22,%22382%22,%22383%22,%22384%22,%22385%22,%22386%22,%22387%22,%22388%22,%22389%22,%22390%22,%22391%22,%22392%22,%22393%22,%22394%22,%22395%22,%22396%22,%22397%22,%22398%22,%22399%22,%22400%22,%22401%22,%22402%22,%22403%22,%22404%22,%22405%22,%22406%22,%22407%22,%22408%22,%22409%22,%22410%22,%22411%22,%22412%22,%22413%22,%22414%22,%22415%22,%22416%22,%22417%22,%22418%22,%22419%22,%22420%22,%22421%22,%22422%22,%22423%22,%22424%22,%22425%22,%22426%22,%22427%22,%22428%22,%22429%22,%22430%22,%22431%22,%22432%22,%22433%22,%22434%22,%22435%22,%22436%22,%22437%22,%22438%22,%22439%22,%22440%22,%22441%22,%22442%22,%22443%22,%22444%22,%22445%22,%22446%22,%22447%22,%22448%22,%22449%22,%22450%22,%22451%22,%22452%22,%22453%22,%22454

In [83]:
active_viewer = active_viewers[1]
viewer_state = ng.to_url(active_viewer.state).replace('neuroglancer-demo','neuroglancer-braincogs')
print(viewer_state)

https://neuroglancer-braincogs.appspot.com#!%7B%22layers%22:%5B%7B%22type%22:%22segmentation%22,%22source%22:%5B%7B%22url%22:%22precomputed://gs://wanglab-pma/princetonmouse_16bit_hierarch_labels%22%7D%5D,%22segments%22:%5B%221119%22,%22378%22,%22379%22,%22380%22,%22381%22,%22382%22,%22383%22,%22384%22,%22385%22,%22386%22,%22387%22,%22388%22,%22389%22,%22390%22,%22391%22,%22392%22,%22393%22,%22394%22,%22395%22,%22396%22,%22397%22,%22398%22,%22399%22,%22400%22,%22401%22,%22402%22,%22403%22,%22404%22,%22405%22,%22406%22,%22407%22,%22408%22,%22409%22,%22410%22,%22411%22,%22412%22,%22413%22,%22414%22,%22415%22,%22416%22,%22417%22,%22418%22,%22419%22,%22420%22,%22421%22,%22422%22,%22423%22,%22424%22,%22425%22,%22426%22,%22427%22,%22428%22,%22429%22,%22430%22,%22431%22,%22432%22,%22433%22,%22434%22,%22435%22,%22436%22,%22437%22,%22438%22,%22439%22,%22440%22,%22441%22,%22442%22,%22443%22,%22444%22,%22445%22,%22446%22,%22447%22,%22448%22,%22449%22,%22450%22,%22451%22,%22452%22,%22453%22,%22454

In [84]:
active_viewer = active_viewers[2]
viewer_state = ng.to_url(active_viewer.state).replace('neuroglancer-demo','neuroglancer-braincogs')
print(viewer_state)

https://neuroglancer-braincogs.appspot.com#!%7B%22layers%22:%5B%7B%22type%22:%22segmentation%22,%22source%22:%5B%7B%22url%22:%22precomputed://gs://wanglab-pma/princetonmouse_16bit_hierarch_labels%22%7D%5D,%22segments%22:%5B%221119%22,%22378%22,%22379%22,%22380%22,%22381%22,%22382%22,%22383%22,%22384%22,%22385%22,%22386%22,%22387%22,%22388%22,%22389%22,%22390%22,%22391%22,%22392%22,%22393%22,%22394%22,%22395%22,%22396%22,%22397%22,%22398%22,%22399%22,%22400%22,%22401%22,%22402%22,%22403%22,%22404%22,%22405%22,%22406%22,%22407%22,%22408%22,%22409%22,%22410%22,%22411%22,%22412%22,%22413%22,%22414%22,%22415%22,%22416%22,%22417%22,%22418%22,%22419%22,%22420%22,%22421%22,%22422%22,%22423%22,%22424%22,%22425%22,%22426%22,%22427%22,%22428%22,%22429%22,%22430%22,%22431%22,%22432%22,%22433%22,%22434%22,%22435%22,%22436%22,%22437%22,%22438%22,%22439%22,%22440%22,%22441%22,%22442%22,%22443%22,%22444%22,%22445%22,%22446%22,%22447%22,%22448%22,%22449%22,%22450%22,%22451%22,%22452%22,%22453%22,%22454

In [10]:
with active_viewers[0].txn() as s:
    print(s)

ViewerState({"dimensions": {"x": [2e-05, "m"], "y": [2e-05, "m"], "z": [2e-05, "m"]}, "position": [139.41319274902344, 519.5, 271], "crossSectionOrientation": [0, 0, -0.7071067690849304, 0.7071067690849304], "crossSectionScale": 1, "projectionOrientation": [0.6734144687652588, 0.7198315262794495, 0.0372951366007328, 0.1642090082168579], "projectionScale": 1024, "layers": [{"type": "segmentation", "source": "precomputed://gs://wanglab-pma/princetonmouse", "objectAlpha": 0.05, "setSegmentsGrayScale": true, "segments": ["1", "10", "100", "1000", "1001", "1002", "1003", "1004", "1005", "1006", "1007", "1008", "1009", "101", "1010", "1011", "1012", "1014", "1015", "1016", "1017", "1018", "1019", "102", "1020", "1021", "1022", "1023", "1024", "1025", "1026", "1027", "1028", "1029", "103", "1030", "1031", "1032", "1033", "1034", "1035", "1036", "1037", "1038", "1039", "104", "1040", "1041", "1042", "1043", "1044", "1045", "1046", "1047", "1048", "1049", "105", "1050", "1051", "1052", "1053", 

In [ ]:
with viewer.txn() as s:
    s.cross_section_scale=2.
    s.projection_scale=1500
    

In [ ]:
with viewer.txn() as s:
    print(s)